<h1>Queries Accompanying Research Questions</h1>

<h2>RQ 1: Do developers look for software recommendations?</h2>

<h2>RQ 2: Can crowdsourced developer knowledge on question answering (Q&A) sites be utilized as wisdom of crowds for software recommendations?</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os
import csv
server = 'DESKTOP-DEK23E9'
database = 'StackExchangeTagsDb_July2024' 
software_recommendations_table = 'softwarerecommendations_998'
stack_overflow_table = '[StackOverflowTagsDb_April2024].[dbo].[StackOverflowTags] '
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()


query=" SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='{}'".format(database)    
print(query)
cursor.execute(query)
all_tables_list = [item[0] for item in cursor.fetchall()]
if software_recommendations_table in all_tables_list:
    all_tables_list.remove(software_recommendations_table)

csvFile = open('tag_overlapping_results.csv', 'w', newline='')
writer = csv.writer(csvFile)
# SR=Software Recommendations, SO=Stack Overflow
writer.writerow(["SiteName", "TotalSiteTags(B)", "SR_Site_Overlap(D+E)", "SR_Site_SO_Overlap(E)"])
for table in all_tables_list: 
    site, count = table.split('_')
    #find overlapping tags between software recommendations and technology based site
    sr_overlap_query=" SELECT count(*) from {} as sr Inner Join  {} as site On site.TagName = sr.TagName ".format(software_recommendations_table, table)    
    cursor.execute(sr_overlap_query)
    sr_overlap_count = cursor.fetchone()[0]
    #find overlapping tags between software recommendations, technology based site and stack overflow 
    so_overlap_query=''' SELECT count(*) from {} as sr 
                        Inner Join {} as site  On site.TagName = sr.TagName 
                        Inner Join  {}  as so On so.TagName = site.TagName '''.format(software_recommendations_table, table,  stack_overflow_table )
    cursor.execute(so_overlap_query)
    so_overlap_count = cursor.fetchone()[0]
    
    print(site)
    writer.writerow([site,count,sr_overlap_count ,so_overlap_count])

csvFile.close()
print('End of comparison')

<h2>RQ 3: Do developers actually look for (any sort of) recommendations on Stack Overflow?</h2>

<p><b>Questions Migrated from Stack Overflow to other Stack Exchange based sites:</b> SQL query for getting questions migrated from Stack Overflow to other Stack Exchange based site.</p>

<p><b>Keywords Based Filtering:</b> We first filtered questions (PostTypeId=1) from Posts based on the presence of any of the keywords in Title, Body or both Title and Body and saved the data in separate tables (<code>Questions_Body_AllKeywords, Questions_Body_AllKeywords, Questions_TitleBodyBoth_AllKeywords</code>) for reducing the query processing time instead of accessing the same data from Posts. <i>Don't forget to create database <code>StackOverflowRecommendationsDb_April2024</code> using SSMS before executing these queries. </i></p>

<p><b>Counting Individual Questions:</b> We then used the following query to find the number of questions containing each of the keywords. Replace the keyword placeholder in the query with any actual keyword.</p>

<p><b>Finding Closed Qs and those with Accepted Answers:</b> We used the following simple SQL queries to find the number of questions containing one or more of the relevant keywords in Title, Body or both which either got closed or received an answer that was accepted by the questioner.</p>

<p><b>Finding Answers and Comments containing relevant keywords</b>: We used the following SQL queries to find the number of answers and comments containing one or more of the relevant keywords in Answers (<code> Posts with PostTypeId=2</code>) and Comments.</p>

<h2>RQ 4: Do developers provide (any sort of) recommendations on Stack Overflow?
</h2>

<p><b>Keywords Based Filtering:</b> We used the following SQL queries to extract <code>Answers</code> from Stack Overflow <code>Posts</code> and subset of <code>Comments</code> which contain one or more recommendation related keywords. 

<h2>RQ 5: Do Stack Overflow discussions containing recommendation related keywords actually provide any kind of recommendations?
</h2>

<p><b>Chronological Analysis of Posts and Comments Filtered by Keywords-Based Approach:</b> We used the following SQL queries to find the number of <code>questions</code>, <code>answers</code> and <code>comments</code> posted on Stack Overflow each year containing one or more of recommendations related keywords.

<p><b>Word Frequency Analysis of Comments:</b> We applied word frequency analysis to comments after performing the following text preprocessing steps.</p>
<li> removing anything enclosed in code blocks </li>
<li> removing html tags </li>
<li> removing urls </li>
<li> removing stop words</li>
<li> removing usernames mentioned in comments starting from @, removing numbers</li>


In [ ]:
import re
from bs4 import BeautifulSoup
import nltk
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
nlp = spacy.load("en_core_web_sm")
tokenizer = ToktokTokenizer()
# nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

df = pd.read_csv('E:\\Publications\\Submitted\\AnalyzingTheWisdomOfCrowdsForSoftwareRecommendations\\Dataset\\CommentsTextOnly.csv')
print(df.head())


In [ ]:
import re
from bs4 import BeautifulSoup
import nltk
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
nlp = spacy.load("en_core_web_sm")
tokenizer = ToktokTokenizer()
# nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

def remove_code_snippet(x):
    result = re.search('<code>(.*)</code>', x)
    if result:
        x= x.replace(result.group(1), '')
    return x

def remove_html_tags(x):
    x = re.sub(r"<.*?>", " ", x)
    return x

df['Text'] = df['Text'].apply(remove_code_snippet)
df['TextClean'] = df['Text'].apply(remove_html_tags)
print('removed code and html')

In [ ]:
import spacy
import nltk
from nltk.corpus import stopwords

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
nltk_stopwords = set(stopwords.words('english'))
additional_stopwords = set(['didnt','dont', 'doesnt', 'isnt', 'arent',   'wasnt' , 'werent' ,'wouldnt', 'shouldnt', 'couldnt', 'mightnt' ,'mustnt', 'shant' ,'wont', 'havent', 'hadnt','hasnt', 'thatll', 'itll',  'its', 'shes', 'hes',  'youd',   'youve', 'youll'])

stopwords= spacy_stopwords.union (nltk_stopwords.union(additional_stopwords))
print(len(stopwords))
print(stopwords)

In [ ]:
from collections import Counter
df['TextCleanWithOutUrls'] = df['TextClean'].str.replace('http\S+|www.\S+', '', case=False)
df['TextCleanTokens'] = df['TextCleanWithOutUrls'].str.split()
tokens_list = df['TextCleanTokens'].tolist()

result = []
for sublist in tokens_list:
    for item in sublist:
        cleanItem= item.lower()
        if not cleanItem.startswith('@'):
            cleanItem = re.sub('[^A-Za-z]+', '', cleanItem)            
            # cleanItem = re.sub('[^A-Za-z0-9]+', '', cleanItem)
        if cleanItem not in stopwords and not cleanItem.startswith('@') and not 'http' in cleanItem:
            result.append(cleanItem)

print(result[:500])
print('---------------')

c = Counter(result)
with open("Comments_WordFrequencyAnalysis", 'w', encoding="utf-8") as f:
    for k,v in  c.most_common():
        f.write( "{} {}\n".format(k,v) )

<h2>Datasets</h2>

<h3>Raw Dataset 1: Recommendation Related Questions with Accepted Answer</h3>

<p>We used the following SQL queries and saved the results in csv format.</p>

<h3>Raw Dataset 2: Recommendation Related Answers and Comments</h3>

<p>We used the following SQL queries and saved the results in csv format.</p>